# Predicitng Which Children Have An Internet Addiction
Kaggle Comp Dec 2024<br>
We know from the outset that the training data is not complete even down to the target 'ssi'. The data also comes in two halves the tabular data in the csv and actigraphy files. So inital plan of attack:<br>
1. Load in both sets of data<br>
2. Data Exploration, inculding detemining how many of our training observations have a actigraphy file.<br>
3. Use KNNs to see if we can't label some of the unlabled observations.<br><br>
We'll be able to make a more detailed plan including considering attribute engineering once we've done a bit of exploration. We'll also decide if/how much we should include the actigraphy files data. <br> 
Before getting into the data, I see that a lot of these attributes focus on phyiscal fitness and health. I suspect these may have little to do with internet addiction. Very fit and active children may be addiction just as much as those that are not so. Also taking into account it much less likely in general for kids to be allowed to spend time out and about without adult supervision you could question 'what else are they supposed to do?'. It also means that kids that don't play organised sport don't have much opportunity of keeping fit and research shows that for many reasons the average American diet is not as healthly as it could be. I suspect that the behaviour questions and sleep disturbance attributes will prove more insightful.<br>
Also like to note that the data is American and therefore they use the term "Fall" instead of the more elegant "Autumn" (as an Australian I am not at all biased about which term is better... it's Autumn). <br> We will use -1 to indicate null values in numeric columns in this exploration part of the process.<br>

In [2]:
#set up
import polars as pl
import plotly.express as px

pl.Config.set_tbl_rows(50);
pl.Config.set_tbl_width_chars(300);
pl.Config.set_tbl_cols(90);

### CSV Data Exploration

In [3]:
#Get the csv data
tableData = pl.read_csv('data/train.csv')

In [4]:
tableData.describe()

statistic,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
str,str,str,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64
"""count""","""3960""","""3960""",3960.0,3960.0,"""2555""",2421.0,"""3310""",3022.0,3027.0,3076.0,898.0,2954.0,2967.0,2954.0,"""1308""",743.0,740.0,740.0,"""3346""",2322.0,2282.0,1074.0,1062.0,1074.0,1063.0,2310.0,2271.0,2305.0,2267.0,2307.0,2269.0,2324.0,2285.0,"""2145""",1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,1991.0,"""475""",475.0,"""1721""",1721.0,"""2736""",2733.0,2734.0,2731.0,2731.0,2729.0,2732.0,2729.0,2730.0,2730.0,2733.0,2734.0,2731.0,2729.0,2732.0,2730.0,2728.0,2725.0,2728.0,2730.0,2733.0,2736.0,"""2618""",2609.0,2606.0,"""3540""",3301.0,2736.0
"""null_count""","""0""","""0""",0.0,0.0,"""1405""",1539.0,"""650""",938.0,933.0,884.0,3062.0,1006.0,993.0,1006.0,"""2652""",3217.0,3220.0,3220.0,"""614""",1638.0,1678.0,2886.0,2898.0,2886.0,2897.0,1650.0,1689.0,1655.0,1693.0,1653.0,1691.0,1636.0,1675.0,"""1815""",1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,1969.0,"""3485""",3485.0,"""2239""",2239.0,"""1224""",1227.0,1226.0,1229.0,1229.0,1231.0,1228.0,1231.0,1230.0,1230.0,1227.0,1226.0,1229.0,1231.0,1228.0,1230.0,1232.0,1235.0,1232.0,1230.0,1227.0,1224.0,"""1342""",1351.0,1354.0,"""420""",659.0,1224.0
"""mean""",null,null,10.433586,0.372727,null,65.454771,null,19.331929,55.946713,89.038615,27.278508,69.648951,81.597236,116.983074,null,4.989233,7.37027,27.581081,null,11.25969,0.476337,22.420438,1.829567,23.518622,1.904045,5.579654,0.330251,8.694924,0.61888,8.805635,0.620097,9.252775,0.785558,null,2.651431,6.719826,19.367048,1237.018187,2064.693747,20.825346,74.021708,15.030554,4.336495,16.85502,1.745354,33.17338,20.02299,67.301883,34.389466,53.998726,null,2.178853,null,2.58955,null,2.370655,2.177762,2.399854,0.839253,2.297545,1.06369,0.586295,1.24652,1.062637,1.304793,1.685443,0.244599,1.340051,1.035505,1.499634,1.452346,1.62789,1.613636,1.158974,0.943652,27.896199,null,41.088923,57.763622,null,1.060588,0.580409
"""std""",null,null,3.574648,0.483591,null,22.341862,null,5.113934,7.473764,44.56904,5.567287,13.611226,13.665196,17.061225,null,2.014072,3.189662,17.707751,null,11.807781,0.499549,10.833995,0.612585,11.148951,0.612344,7.390161,0.470407,3.429301,0.485769,3.422167,0.485469,2.988863,0.410525,null,1.028267,92.586325,5.047848,1872.383246,2836.246272,73.266287,199.433753,5.792505,6.356402,199.372119,0.68

<br>
Worth noting here that there are only 4 columns that have no nulls: id, Basic_Demos-Enroll_Season (Season of Enrollment), Basic_Demos-Age (Age) & Basic_Demos_Sex (Sex).<br>
I'm going to predict now that those attributes alone will be wildly insufficent to determine internet addiction.<br>
But for now, let's get a better look at the state of our csv data vis-a-vis missing data. We know there are 83 columns including id so let's is a histogram looking at number of columns missing.

In [5]:
nullCount = tableData.transpose().with_columns(pl.all().null_count()).unique().transpose().rename({'column_0':'NullCount'})

fig = px.histogram(nullCount.to_pandas(),
                   x='NullCount',
                   title='Histogram of Missing Values Per Row')
fig.show()

<br>Glad to see that there are a good number of obsrevations that have low numbers of missing values and there is still a significant number that are almost all nulls. The likely hood of been able to determine much for these rows are fairly low.

In [71]:
#Straight histogram function
def histogram(data,x, title, stacked=None):
    
    fig = px.histogram(data.to_pandas(),
                       x=x,
                       title=title,
                       color=stacked)
    
    fig.show()
    
#function to plot histograms facted by season of participation
def histogramSeason(data, x, facet, title, stacked=None):

    fig = px.histogram(data.fill_null(-1).to_pandas(),
                       x=x,
                       facet_col=facet,
                       facet_col_wrap=2,
                       title = title,
                       color=stacked,
                       height=800)
    fig.update_xaxes(matches=None)
    fig.update_yaxes(matches=None)
    
    fig.show()

#function to plot scatter plots facted by season of participation
def scatterSeason(data, x, y, facet, title):

    fig = px.scatter(data.fill_null(-1).to_pandas(),
                x=x,
                y=y,
                facet_col=facet,
                facet_col_wrap=2,
                title=title,
                height=800)
    fig.update_xaxes(matches=None)
    fig.update_yaxes(matches=None)
    fig.show()

#function to plot a scatter matrix coloured by season of participation
def scatterMatrixSeason(data, dimensions, title, colour):
    fig = px.scatter_matrix(data.fill_null(-1).to_pandas(),
                            title=title,
                            dimensions=dimensions,
                            color=colour,
                            height=1500)
    fig.update_traces(diagonal_visible=False)
    fig.show()

##### CSV Data Exploration: Demographics

In [24]:
#Season of enrollment
histogram(tableData.select('Basic_Demos-Enroll_Season'), x='Basic_Demos-Enroll_Season', title = 'Histogram of Season of Enrollment')

In [25]:
#Age 
histogram(tableData, x='Basic_Demos-Age', title='Histogram of Age')

In [26]:
#Sex
histogram(tableData, x='Basic_Demos-Sex', title='Histogram of Sex')

0 = male <br>
1 = female <br>
Can't say I'm too suprised by their been approx. 1000 more males than females.

In [27]:
tableData.group_by('sii').len().sort('sii')

sii,len
i64,u32
null,1224
0,1594
1,730
2,378
3,34


Note quite high number of observations that have no sii category.

#### CSV Data Exploration: Children's Global Assessment Scale

In [28]:
histogramSeason(tableData, x='CGAS-CGAS_Score', facet='CGAS-Season', title='Histogram of CGAS Score')

Considering the range of a CGAS is between either 1-90 or 1-100 (and based on the other avaliable points, I am going with the latter in this case), we have an outlier in winter. Most likely a type-o so it will be changed 999 -> 99. <br>
Otherwise the histograms across the seasons appear quite similar.

In [33]:
scatterSeason(tableData, y='CGAS-CGAS_Score', x='sii', facet='CGAS-Season', title='Scatter Plot of CGAS Socre vs. sii')

Appears to be a slight decline in max CGAS Score as the internet addiction gets more serious but they do overlap for the most part which will limit it's usefullness.

#### CSV Data Exploration: Physical Measurements

In [49]:
scatterMatrixSeason(tableData, ['Physical-BMI',
                                'Physical-Height',
                                'Physical-Weight',
                                'Physical-Waist_Circumference',
                                'Physical-Diastolic_BP',
                                'Physical-HeartRate',
                                'Physical-Systolic_BP',
                                'sii'],
                    title = 'Scatter Matrix of Physical Measurements',
                    colour='Physical-Season')

It appears that physical measurements have little to do with internet addiction.

#### CSV Data Exploration: Fitness Gram Vitals & Treadmill

In [51]:
scatterMatrixSeason(tableData,
                    dimensions=['Fitness_Endurance-Max_Stage',
                                'Fitness_Endurance-Time_Mins',
                                'Fitness_Endurance-Time_Sec',
                                'sii'],
                    title = 'Scatter Matrix for Fitness Gram Vitals & Treadmill', 
                    colour='Fitness_Endurance-Season')

A few suspect points and kudos to the kid that apparently got the all the way to stage 28 in 8 min and 12 secs!

#### CSV Data Exploration: Fitness Gram Vitals

In [59]:
scatterMatrixSeason(tableData, ['FGC-FGC_CU',
                                'FGC-FGC_GSND',
                                'FGC-FGC_PU',
                                'FGC-FGC_SRL',
                                'FGC-FGC_SRR',
                                'FGC-FGC_TL',
                                'sii'],
                    title='Scatter Matrix of Fitness Gram Totals',
                    colour='FGC-Season')

Even when looking at the above plot when filtering at age level, there remains no pattern or clear realtionship between any of these measures and sii. The spread and location of the scatters slightly shift as you would expect but still the same shapes.

#### CSV Data Exploration: Bio-electric Impedance Analysis

In [63]:
# 1=Very Light, 2=Light, 3=Moderate, 4=Heavy, 5=Exceptional
histogramSeason(tableData, x='BIA-BIA_Activity_Level_num', facet='BIA-Season', title='Histogram of Activity Level with stacked sii', stacked='sii')

In [ ]:
# 1=Small, 2=Medium, 3=Large
histogramSeason(tableData, x='BIA-BIA_Frame_num', facet='BIA-Season', title='Histogram of Body Frame with stacked sii', stacked='sii')

In [68]:
scatterMatrixSeason(tableData,
                    ['BIA-BIA_BMC',
                     'BIA-BIA_BMI',
                     'BIA-BIA_BMR',
                     'BIA-BIA_DEE',
                     'BIA-BIA_ECW',
                     'BIA-BIA_FFM',
                     'BIA-BIA_FFMI',
                     'BIA-BIA_FMI',
                     'sii'],
                     title='Scatter Matrix of Numeric Data from Bio-electric Impedence Analysis Part 1',
                     colour='BIA-Season')

In [69]:
scatterMatrixSeason(tableData,
                    ['BIA-BIA_Fat',
                     'BIA-BIA_ICW',
                     'BIA-BIA_LDM',
                     'BIA-BIA_LST',
                     'BIA-BIA_SMM',
                     'BIA-BIA_TBW',
                     'sii'],
                     title='Scatter Matrix of Numeric Data from Bio-electric Impedence Analysis Part 2',
                     colour='BIA-Season')


#### CSV Data Exploration: Parent-Child Addiction Test

In [70]:
scatterSeason(tableData, x='sii', y='PCIAT-PCIAT_Total', facet='PCIAT-Season', title='Parent-Child Addiction Test Totals')

As expected the behaviour related attributes are showing a very clear distiction with level of internet addicition, granted it is a test to determine internet addition.<br>
We'll look at all the questions, all where a multiple choice questions with the same scale:<br>
0=Does Not Apply, 1=Rarely, 2=Occasionally, 3=Frequently, 4=Often, 5=Always

In [72]:
histogram(tableData, x='PCIAT-PCIAT_01', title="How often does your child disobey time limits you set for online use?", stacked='sii')

In [74]:
histogram(tableData, x='PCIAT-PCIAT_02', title="How often does your child neglect household chores to spend more time online?", stacked='sii')

In [75]:
histogram(tableData, x='PCIAT-PCIAT_03', title="How often does your child prefer to spend time online rather than with the rest of your family?", stacked='sii')

In [77]:
#Strongly suggest this one is not an accurate reflection of the truth, how many kids are going to be honest about this and how many parents are suprivising their use to actually know
histogram(tableData, x='PCIAT-PCIAT_04', title="How often does your child form new relationships with fellow online users?", stacked='sii')

In [78]:
histogram(tableData, x='PCIAT-PCIAT_05', title="How often do you complain about the amount of time your child spends online?", stacked='sii')

In [79]:
histogram(tableData, x='PCIAT-PCIAT_06', title="How often do your child's grades suffer because of the amount of time he or she spends online?", stacked='sii')


In [80]:
histogram(tableData, x='PCIAT-PCIAT_07', title="How often does your child check his or her e-mail before doing something else?", stacked='sii')

In [81]:
histogram(tableData, x='PCIAT-PCIAT_08', title="How often does your child seem withdrawn from others since discovering the Internet?", stacked='sii')

In [82]:
histogram(tableData, x='PCIAT-PCIAT_09', title="How often does your child become defensive or secretive when asked what he or she does online?", stacked='sii')

In [83]:
histogram(tableData, x='PCIAT-PCIAT_10', title="How often have you caught your child sneaking online against your wishes?", stacked='sii')

In [84]:
histogram(tableData, x='PCIAT-PCIAT_11', title="How often does your child spend time along in his or her room playing on the computer?", stacked='sii')

In [85]:
histogram(tableData, x='PCIAT-PCIAT_12', title='How often does your child receive strange phone calls from new "online" friends?', stacked='sii')

In [86]:
histogram(tableData, x='PCIAT-PCIAT_13', title='How often does your child snap, yell, or act annoyed if bothered while online?', stacked='sii')

In [87]:
histogram(tableData, x='PCIAT-PCIAT_14', title='How often does your child seem more tired and fatigued than he or she did before the Internet came along?', stacked='sii')

In [88]:
histogram(tableData, x='PCIAT-PCIAT_15', title='How often does your child seem preoccupied with being back online when off-line?', stacked='sii')

In [89]:
histogram(tableData, x='PCIAT-PCIAT_16', title='How often does your child throw tantrums with your interference about how long he or she spends online?', stacked='sii')

In [90]:
histogram(tableData, x='PCIAT-PCIAT_17', title='How often does your child choose to spend time online rather than doing once enjoyed hobbies and/or outside interests?', stacked='sii')


In [91]:
histogram(tableData, x='PCIAT-PCIAT_18', title='How often does your child become angry or belligerent when your place time limits on how much time he or shes is allowed to spend online?', stacked='sii')

In [92]:
histogram(tableData, x='PCIAT-PCIAT_19', title='How often does your child choose to spend more time online than going out with friends?', stacked='sii')

In [93]:
histogram(tableData, x='PCIAT-PCIAT_20', title='How often does your child feel depressed, moody, or nervous when off-line which seems to go away once back online?', stacked='sii')

So it seems to be a combination of the questions that can indicate fairly well if there is a internet addition problem though the questionaire appears to be a little dated.

#### CSV Data Exploration: Sleep Disturbance Scale

In [95]:
scatterMatrixSeason(tableData,
                    ['SDS-SDS_Total_Raw',
                     'SDS-SDS_Total_T',
                     'sii'],
                     title='Scatter Matrix for Sleep Disturbance Scale',
                     colour='SDS-Season')

#### CSV Data Exploration: Internet Use
Uses categories<br>
0=Less than 1h/day, 1=Around 1h/day, 2=Around 2hs/day, 3=More than 3hs/day

In [97]:
histogramSeason(tableData,
              'PreInt_EduHx-computerinternet_hoursday',
              title='Internet Use In Hours',
              facet='PreInt_EduHx-Season',
              stacked='sii')

Interestingly there appears to be no relationship between time using the internet and been addicted.

## Arctigraphy Files Data Exploration
Hereafter to be know as af data exploration.